In [1]:
from copy import deepcopy
import os
import warnings

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import fastplotlib as fpl

import mesmerize_core as mc
import mesmerize_viz

2024-07-18 21:57:58.270966: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 21:57:58.271001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered


In [2]:
def is_saved_tiff(tiff_path):
    if Path(tiff_path).is_file():
        return True
    else:
        return False

In [3]:
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache
if os.name == "nt":
    # disable the cache on windows
    cnmf_cache.set_maxsize(0)

In [4]:
pd.options.display.max_colwidth = 120

In [6]:
from pathlib import Path
import scanreader
from tifffile import imwrite


### Set up data and save-paths

In [7]:
parent = Path('/home/rbo/caiman_data')
raw_tiff_name = parent / 'high_res.tif'

save =  parent / 'v1'
savename = save / 'extracted_plane_1.tif'

reader = scanreader.read_scan(str(raw_tiff_name), join_contiguous=True)

### Save as a multi-page tiff

In [ ]:
for plane in range(0, reader.num_channels):
    savename = save / f"extracted_plane_{plane + 1}.tiff"
    print(savename)
    if not savename.is_file():
        print(f"Saving: {savename}\n")
        imwrite(savename, reader[:,:,:,plane,:].squeeze().transpose(2,0,1), photometric='minisblack')
    else:
        print(f"This file already exists in save path: {savename}\n")

In [11]:
mc.set_parent_raw_data_path("/home/rbo/caiman_data/v1/")

PosixPath('/home/rbo/caiman_data/v1')

In [12]:
batch_path = mc.get_parent_raw_data_path().joinpath("mesmerize-batch/batch.pickle")

In [13]:
# create a new batch
try:
    df = mc.create_batch(batch_path)
except FileExistsError:
    df = mc.load_batch(batch_path)

In [14]:
movie_path = mc.get_parent_raw_data_path().joinpath("extracted_plane_0.tiff")

In [15]:
mcorr_params1 =\
{
  'main': # this key is necessary for specifying that these are the "main" params for the algorithm
    {
        'max_shifts': [6, 6],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

In [16]:
# add an item to the DataFrame
df.caiman.add_item(
    algo='mcorr',
    input_movie_path=movie_path,
    params=mcorr_params1,
    item_name=movie_path.stem,  # filename of the movie, but can be anything
)

df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,extracted_plane_0,extracted_plane_0.tiff,"{'main': {'max_shifts': (6, 6), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': d70fc97b-b414-4f50-b83c-57365d8a11e4/d70fc97b-b414-4f50-b83c-57365d8a11e4_mean_projection.n...,2024-07-18T21:43:39,2024-07-18T21:45:12,43.55 sec,None,d70fc97b-b414-4f50-b83c-57365d8a11e4
1,mcorr,extracted_plane_0,extracted_plane_0.tiff,"{'main': {'max_shifts': (6, 6), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan': '...",None,2024-07-18T21:58:49,None,None,None,b1182bfa-f176-4d84-bf2a-bd8874005189


In [17]:
df.iloc[0].caiman.run()

starting mc


<Popen: returncode: 0 args: '/home/rbo/caiman_data/v1/mesmerize-batch/d70fc9...>

In [18]:
df = df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,extracted_plane_0,extracted_plane_0.tiff,"{'main': {'max_shifts': (6, 6), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': d70fc97b-b414-4f50-b83c-57365d8a11e4/d70fc97b-b414-4f50-b83c-57365d8a11e4_mean_projection.n...,2024-07-18T21:43:39,2024-07-18T21:59:32,40.22 sec,None,d70fc97b-b414-4f50-b83c-57365d8a11e4
1,mcorr,extracted_plane_0,extracted_plane_0.tiff,"{'main': {'max_shifts': (6, 6), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan': '...",None,2024-07-18T21:58:49,None,None,None,b1182bfa-f176-4d84-bf2a-bd8874005189


In [19]:
index = 0 # we will fetch stuff from index 0 which we just ran

# get the motion corrected movie memmap
mcorr_movie = df.iloc[0].mcorr.get_output()
mcorr_movie.shape

# path to the mcorr memmap if you ever need it
mcorr_memmap_path = df.iloc[0].mcorr.get_output_path()
mcorr_memmap_path

# mean projection, max and std projections are also available
mean_proj = df.iloc[0].caiman.get_projection("mean")
mean_proj.shape

# the input movie, note that we use `.caiman` here instead of `.mcorr`
input_movie = df.iloc[0].caiman.get_input_movie()
input_movie.shape

(1730, 600, 576)

In [ ]:
plt.imshow(mean_proj)
plt.show()

In [ ]:
for name, value in os.environ.items():
    print("{0}: {1}".format(name, value))

In [20]:
viz = df.mcorr.viz(data_options=["input", "mcorr", "mean", "corr"])

/home/rbo/miniconda3/envs/caiman/lib/python3.11/site-packages/ipydatagrid/datagrid.py:512: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

/home/rbo/miniconda3/envs/caiman/lib/python3.11/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


In [ ]:
viz.close()

In [ ]:
mcorr_params2 =\
{
  'main':
    {
        'max_shifts': [4, 4],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# add other param variant to the batch
df.caiman.add_item(
  algo='mcorr',
  item_name=movie_path.stem,
  input_movie_path=movie_path,
  params=mcorr_params2
)

df

In [ ]:
# copy the mcorr_params2 dict to make some changes
new_params = deepcopy(mcorr_params2)

# some variants of max_shifts
for shifts in [2, 32]: 
    for strides in [12, 24, 64]:
        overlaps = int(strides / 2)
        # deep copy is the safest way to copy dicts
        new_params = deepcopy(new_params)

        # assign the "max_shifts"
        new_params["main"]["max_shifts"] = (shifts, shifts)
        new_params["main"]["strides"] = (strides, strides)
        new_params["main"]["overlaps"] = (overlaps, overlaps)

        df.caiman.add_item(
          algo='mcorr',
          item_name=movie_path.stem,
          input_movie_path=movie_path,
          params=new_params
        )

In [ ]:
diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])
diffs

In [ ]:
for i, row in df.iterrows():
    if row["outputs"] is not None: # item has already been run
        continue # skip
        
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

In [ ]:
df = df.caiman.reload_from_disk()

In [ ]:
%gui qt